### 1. Imports + paths + localizar inputs

In [ ]:
from pathlib import Path
import sys
import shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

NOTEBOOK_DIR = Path.cwd().resolve()
for p in [NOTEBOOK_DIR] + list(NOTEBOOK_DIR.parents):
    if (p / "src").exists():
        if str(p) not in sys.path:
            sys.path.insert(0, str(p))
        break
else:
    raise FileNotFoundError(
        "No encontré carpeta 'src' subiendo desde el CWD. "
        "Ejecuta el notebook desde dentro del repo."
    )

from src.paths import project_paths

P = project_paths(Path.cwd())
PROJECT_ROOT = P["PROJECT_ROOT"]

# Carpetas
FIGURES_DIR = P["FIGURES_DIR"]
RESULTS_DIR = P["RESULTS_DIR"]

# Compatibilidad con runs "legacy" solo para localizar inputs
FIG_DIR_CANDIDATES = [
    FIGURES_DIR,
    FIGURES_DIR / "final",
    PROJECT_ROOT / "figures_final",   # legacy
]

SUM_DIR_CANDIDATES = [
    RESULTS_DIR / "summary_tables_final",
    RESULTS_DIR / "summary_tables",
    PROJECT_ROOT / "summary_tables_final",  # legacy
]

OUT_DIR = PROJECT_ROOT / "final_deliverables"
OUT_DIR.mkdir(exist_ok=True)

print("NOTEBOOK_DIR :", NOTEBOOK_DIR)
print("PROJECT_ROOT :", PROJECT_ROOT)
print("FIG_DIR candidates:")
for d in FIG_DIR_CANDIDATES:
    print(" -", d, "(exists=", d.exists(), ")")
print("SUM_DIR candidates:")
for d in SUM_DIR_CANDIDATES:
    print(" -", d, "(exists=", d.exists(), ")")
print("OUT_DIR      :", OUT_DIR)

def first_existing(paths):
    for p in paths:
        if p is not None and Path(p).exists():
            return Path(p)
    return None

# Resolver volcanos FOR_FIGURE Mono y T
mono_name_candidates = [
    "Volcano_pseudobulk_Mono_Healthy_vs_Cirrhosis_FOR_FIGURE.png",                     # legacy
    "Volcano_pseudobulk_Level1_refined_Mono_Healthy_vs_Cirrhosis_FOR_FIGURE.png",
]

t_name_candidates = [
    "Volcano_pseudobulk_T_Healthy_vs_Cirrhosis_FOR_FIGURE.png",                        # legacy
    "Volcano_pseudobulk_Level1_refined_T_Healthy_vs_Cirrhosis_FOR_FIGURE.png",
]

pMono_candidates = []
pT_candidates = []
for d in FIG_DIR_CANDIDATES:
    for fn in mono_name_candidates:
        pMono_candidates.append(d / fn)
    for fn in t_name_candidates:
        pT_candidates.append(d / fn)

pMono = first_existing(pMono_candidates)
pT = first_existing(pT_candidates)

if pMono is None or pT is None:
    print("\n[ERROR] No pude localizar los volcanos requeridos.")
    print("Busqué Mono en:")
    for p in pMono_candidates:
        print(" -", p)
    print("Busqué T en:")
    for p in pT_candidates:
        print(" -", p)
    raise FileNotFoundError("Faltan volcanos FOR_FIGURE (Mono y/o T). Revisa nombres o carpeta de figuras.")

print("\n[OK] Volcano Mono:", pMono)
print("[OK] Volcano T   :", pT)

### 2. Ensamblar SuppFigS1 y guardar PNG

In [ ]:
imgM = mpimg.imread(pMono)
imgT = mpimg.imread(pT)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, img, lab, ttl in [
    (axes[0], imgM, "A", "Mono: Healthy vs Cirrhosis (pseudobulk, por paciente)"),
    (axes[1], imgT, "B", "T: Healthy vs Cirrhosis (pseudobulk, por paciente)"),
]:
    ax.imshow(img)
    ax.axis("off")
    ax.text(
        0.01, 0.99, lab,
        transform=ax.transAxes,
        va="top", ha="left",
        fontsize=14, fontweight="bold"
    )
    ax.set_title(ttl, fontsize=10)

plt.tight_layout()

# Guardado: se guarda junto a los volcanos detectados
out_supp_dir = pMono.parent
out_supp = out_supp_dir / "SuppFigS1_Volcanos_pseudobulk_Mono_T.png"

fig.savefig(out_supp, dpi=300, bbox_inches="tight")
plt.close(fig)

print("Saved:", out_supp)

### 3. Copiar a final_deliverables/

In [ ]:
def find_in_sumdir_candidates(relname_list):
    """
    Devuelve el primer archivo existente entre varias carpetas SUM_DIR_CANDIDATES
    y varias opciones de filename.
    """
    paths = []
    for d in SUM_DIR_CANDIDATES:
        for fn in relname_list:
            paths.append(d / fn)
    return first_existing(paths), paths

# Tablas soporte: nombres posibles (para no romper si cambió sufijo by_target/by_celltype)
tableS1_candidates = [
    "DE_pseudobulk_FOR_REPORT_LINFO_CLEAN_top10_by_celltype.csv",     # legacy
    "DE_pseudobulk_Level2_final_FOR_REPORT_LINFO_CLEAN_top10_by_target.csv",
    "DE_pseudobulk_Level1_refined_FOR_REPORT_LINFO_CLEAN_top10_by_target.csv",
]

# scCODA Level1_refined
tableS2_candidates = [
    "scCODA_Level1refined_credible_effects_ref-T.csv",  # nombre esperado
]

tableS1_src, tableS1_tried = find_in_sumdir_candidates(tableS1_candidates)
tableS2_src, tableS2_tried = find_in_sumdir_candidates(tableS2_candidates)

to_copy = {
    "SuppFigS1_Volcanos_pseudobulk_Mono_T.png": out_supp,
    "SuppFigS1A_Volcano_pseudobulk_Mono.png": pMono,
    "SuppFigS1B_Volcano_pseudobulk_T.png": pT,
}

# Añadir tablas si existen
if tableS1_src is not None:
    to_copy["TableS1_DE_pseudobulk_top10_reporting.csv"] = tableS1_src
else:
    print("\n[WARN] No encontré TableS1 candidata. Busqué en:")
    for p in tableS1_tried:
        print(" -", p)

if tableS2_src is not None:
    to_copy["TableS2_scCODA_Level1refined_credible_effects_ref-T.csv"] = tableS2_src
else:
    print("\n[WARN] No encontré TableS2 candidata. Busqué en:")
    for p in tableS2_tried:
        print(" -", p)

print("\nCopiando a final_deliverables/ ...")
for name, src in to_copy.items():
    src = Path(src)
    if not src.exists():
        print("[SKIP missing]", name, "->", src)
        continue
    dst = OUT_DIR / name
    shutil.copy2(src, dst)
    print("[OK]", dst)

print("\n[OK] DE (pseudobulk) cerrado como suplementaria.")
print("Contenido final_deliverables (resumen):")
for p in sorted(OUT_DIR.glob("*")):
    print(" -", p.name)